In [1]:
from jetstream_hugo.definitions import *
from jetstream_hugo.plots import *
from jetstream_hugo.data import *
# from jetstream_hugo.anyspell import *
from jetstream_hugo.jet_finding import *
from jetstream_hugo.clustering import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

# create jet relative climatologies

In [3]:
exp = JetFindingExperiment(DataHandler(f"{DATADIR}/ERA5/plev/high_wind/6H/results/1"))

In [4]:
def create_jet_relative_clim(exp, da):
    all_jets_one_df = exp.find_jets()
    jets = all_jets_one_df.with_columns(pl.col("time").dt.round("1d"))
    jets = jets.with_columns(jets.group_by("time", maintain_order=True).agg(pl.col("jet ID").rle_id())["jet ID"].explode())
    indexer = iterate_over_year_maybe_member(jets, da)
    to_average = []
    for idx1, idx2 in tqdm(indexer, total=len(YEARS)):
        jets_ = jets.filter(*idx1)
        da_ = da.sel(**idx2)
        try:
            jets_with_interp = gather_normal_da_jets(jets_, da_, half_length=20)
        except (KeyError, ValueError):
            break
        varname = da_.name + "_interp"
        jets_with_interp = interp_jets_to_zero_one(jets_with_interp, [varname, "is_polar"])
        jets_with_interp = jets_with_interp.group_by("time", pl.col("is_polar") > 0.5, "norm_index", "n", maintain_order=True).agg(pl.col(varname).mean() )
        to_average.append(jets_with_interp)
    to_average = pl.concat(to_average)
    clim = to_average.group_by(pl.col("time").dt.ordinal_day().alias("dayofyear"), "is_polar", "norm_index", "n").agg(pl.col(varname).mean()).sort("dayofyear", "is_polar", "norm_index", "n")
    clim_ds = polars_to_xarray(clim, ["dayofyear", "is_polar", "n", "norm_index"])
    clim_ds.to_netcdf(exp.path.joinpath(f"{da.name}_relative_clim.nc"))

In [5]:
compute_all_smoothed_anomalies("ERA5", "surf", "t2m", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)
compute_all_smoothed_anomalies("ERA5", "surf", "t2m", "6H", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
da_t2m = open_da("ERA5", "surf", "t2m", "dailymean", "all")
create_jet_relative_clim(exp, da_t2m)

 33%|███▎      | 21/64 [05:06<10:34, 14.76s/it]

In [10]:
da_tp = open_da("ERA5", "surf", "tp", "dailysum", "all")
create_jet_relative_clim(exp, da_tp)

100%|██████████| 64/64 [15:14<00:00, 14.28s/it]


In [11]:
da_apvs = open_da("ERA5", "thetalev", "apvs", "dailymean", "all", levels=350)
create_jet_relative_clim(exp, da_apvs)

100%|██████████| 64/64 [21:16<00:00, 19.94s/it]


In [12]:
da_cpvs = open_da("ERA5", "thetalev", "cpvs", "dailymean", "all", levels=350)
create_jet_relative_clim(exp, da_cpvs)

100%|██████████| 64/64 [19:48<00:00, 18.56s/it]


# arco-era5 tests

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

base_ds = standardize(ar_full_37_1h["total_precipitation"].chunk("auto"))
base_ds = (
    base_ds
    .sel(
        lat=base_ds.lat >= 0,
        time=np.isin(base_ds.time.dt.year, YEARS)
    )
    .isel(lon=slice(None, None, 2), lat=slice(None, None, 2))
)

six_hourly = base_ds.resample(time="6h").sum()
daily = six_hourly.resample(time="1d").sum()
six_hourly = six_hourly * 4

base_path_1 = Path(f"{DATADIR}/ERA5/surf/tp/6H")
base_path_1.mkdir(exist_ok=True, parents=True)
base_path_2 = Path(f"{DATADIR}/ERA5/surf/tp/dailysum")
base_path_2.mkdir(exist_ok=True, parents=True)
for year in YEARS:
    opath_1 = base_path_1.joinpath(f"{year}.nc")
    opath_2 = base_path_2.joinpath(f"{year}.nc")
    if not opath_1.is_file():
        six_hourly_ = compute(six_hourly.sel(time=six_hourly.time.dt.year == year), progress_flag=True)
        six_hourly_.to_netcdf(opath_1)
    if not opath_2.is_file():
        daily_ = compute(daily.sel(time=daily.time.dt.year == year), progress_flag=True)
        daily_.to_netcdf(opath_2)
    print(f"Completed {year}")

In [17]:
six_hourly = base_ds.resample(time="6h").sum()
daily = six_hourly.resample(time="1d").sum()
six_hourly = six_hourly * 4

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full)

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full)

orig_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean")
base_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean_2")
for year in tqdm(YEARS):
    for month in trange(1, 13, leave=False):
        month_str = str(month).zfill(2)
        opath = base_path.joinpath(f"{year}{month_str}.nc")
        if opath.is_file():
            continue
        ipath = orig_path.joinpath(f"{year}{month_str}.nc")
        ds = xr.open_dataset(ipath)
        this_temp = temp_full.sel(time=ds.time.values, lev=ds["lev"])
        this_temp = this_temp * (1000 / this_temp.lev) ** KAPPA
        this_temp = this_temp.reset_coords("lev", drop=True)
        ds["theta"] = compute(this_temp, progress_flag=True)
        ds.to_netcdf(opath)

# new pvs das

# CESM

### new download with urls

In [ ]:
from collections.abc import Sequence
from itertools import pairwise
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from jetstream_hugo.definitions import DATADIR, KAPPA, compute
from jetstream_hugo.data import standardize, flatten_by, extract
import numpy as np 
from netCDF4 import Dataset
import xarray as xr
import argparse
import traceback


varname_to_search_dict = {
    "high_wind": ["U", "V", "T"],
    "mid_wind": ["U500", "V500"],
    "PRECL": "PRECL",
    "TS": "TS",
}
levels_dict = {
    "high_wind": list(range(14, 18)),
    "mid_wind": "all",
    "PRECL": "all",
    "TS": "all",
}
experiment_dict = {
    "past": "BHISTcmip6",
    "future": "BSSP370cmip6",
}
yearbounds = {
    "past": np.arange(1960, 2021, 10),
    "future": np.arange(2045, 2106, 10),
}
yearbounds["past"][-1] = yearbounds["past"][-1] - 5
yearbounds["future"][-1] = yearbounds["future"][-1] - 4
timebounds = {key: [f"{year1}0101-{year2 - 1}1231" for year1, year2 in pairwise(val)] for key, val in yearbounds.items()}

members = [f"{year}.{str(number).zfill(3)}" for year, number in zip(range(1001, 1201, 20), range(1, 11))]
for startyear in [1231, 1251, 1281, 1301]:
    members.extend(f"{startyear}.{str(number).zfill(3)}" for number in range(1, 11))
    
members2 = [f"r{number}i{year}p1f1" for year, number in zip(range(1001, 1201, 20), range(1, 11))]
for startyear in [1231, 1251, 1281, 1301]:
    members2.extend(f"r{number}i{startyear}p1f1" for number in range(1, 11))
    
season = None
minlon = -180
maxlon = 180
minlat = 0
maxlat = 90
    
    
def get_url(varname: str, period: str, member: str, timebounds: str):
    experiment = experiment_dict[period]
    h = 6 if varname in ["U", "V", "T"] else 1

    return fr"https://tds.ucar.edu/thredds/fileServer/datazone/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/{varname}/b.e21.{experiment}.f09_g17.LE2-{member}.cam.h{h}.{varname}.{timebounds}.nc?api-token=ayhBFVYTOtGi2LM2cHDn6DjFCoKeCAqt69z8Ezt4#mode=bytes"


var = "T"
member = members[0]
period = "future"
da = []
for tb in timebounds["future"]:
    da.append(
        standardize(xr.open_dataset(
            get_url(var, period, member,tb),            
            engine="h5netcdf"
        )[var])
    )
da = xr.concat(da, "time")

In [20]:
print(members2[0], member)
da

r1i1001p1f1 1001.001


<xarray.DataArray 'T' (time: 20441, lev: 32, lat: 192, lon: 288)> Size: 145GB
dask.array<concatenate, shape=(20441, 32, 192, 288), dtype=float32, chunksize=(1, 16, 96, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lev      (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * time     (time) object 164kB 2045-01-01 00:00:00 ... 2101-01-01 00:00:00
  * lon      (lon) float64 2kB -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
Attributes:
    mdims:         1
    units:         K
    long_name:     Temperature
    cell_methods:  time: mean

In [24]:
da.time

<xarray.DataArray 'time' (time: 20441)> Size: 164kB
array([cftime.DatetimeNoLeap(2045, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2045, 1, 2, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2045, 1, 3, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2100, 12, 30, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2100, 12, 31, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2101, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 164kB 2045-01-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    long_name:  time
    bounds:     time_bnds

In [25]:
ds.time

<xarray.DataArray 'time' (time: 14600)> Size: 117kB
array([cftime.DatetimeNoLeap(2060, 1, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2060, 1, 2, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2060, 1, 3, 12, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2099, 12, 29, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2099, 12, 30, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2099, 12, 31, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
    member   <U11 44B 'r1i1001p1f1'
  * time     (time) object 117kB 2060-01-01 12:00:00 ... 2099-12-31 12:00:00
Attributes:
    bounds:     time_bnds
    long_name:  time

In [23]:
da.sel(time=ds.time, lon=ds.lon, lat=ds.lat)

KeyError: "not all values found in index 'time'"

In [ ]:
ds = xr.open_mfdataset(Path("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/high_wind/ssp370").glob(f"{members2[0]}-????.nc"))

In [22]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 14600, lat: 69, lon: 97, nbnd: 2)
Coordinates:
    member     <U11 44B 'r1i1001p1f1'
  * lat        (lat) float32 276B 15.55 16.49 17.43 18.38 ... 77.75 78.69 79.63
    time_bnds  (time, nbnd) object 234kB dask.array<chunksize=(365, 2), meta=np.ndarray>
  * time       (time) object 117kB 2060-01-01 12:00:00 ... 2099-12-31 12:00:00
  * lon        (lon) float32 388B -80.0 -78.75 -77.5 -76.25 ... 37.5 38.75 40.0
Dimensions without coordinates: nbnd
Data variables:
    u          (time, lat, lon) float32 391MB dask.array<chunksize=(365, 69, 97), meta=np.ndarray>
    lev        (time, lat, lon) float32 391MB dask.array<chunksize=(365, 69, 97), meta=np.ndarray>
    v          (time, lat, lon) float32 391MB dask.array<chunksize=(365, 69, 97), meta=np.ndarray>
    s          (time, lat, lon) float32 391MB dask.array<chunksize=(365, 69, 97), meta=np.ndarray>
Attributes: (12/20)
    Conventions:                       CF-1.0
    host:                              mom1
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  day_1
    ...                                ...
    intake_esm_attrs:start_time:       2015-01-01 12:00:00
    intake_esm_attrs:end_time:         2100-12-31 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.ssp370.daily.cmip6
    orig_lev:                          [197.90809 232.82862 273.91082 322.2419 ]
    flattened:                         1

### new cesm zarrification

In [3]:
basepath = Path("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/high_wind/ssp370")
paths = list(basepath.glob("*.nc"))
names = [path.stem.split("-") for path in paths]
members = [name[0] for name in names]
years = [name[1] for name in names]
for i, member in enumerate(tqdm(np.unique(members))):
    da = xr.open_mfdataset(basepath.joinpath(f"{member}-*.nc").as_posix())
    kwargs = {"mode": "w"} if i == 0 else {"mode": "a", "append_dim": "member"}
    da["member"] = da["member"].astype("<U15")
    da = da.expand_dims("member").copy(deep=True)
    break
    # da.to_zarr(basepath.joinpath("ds.zarr"), **kwargs)

  0%|          | 0/50 [00:03<?, ?it/s]


In [4]:
da

<xarray.Dataset> Size: 2GB
Dimensions:    (member: 1, time: 14600, lat: 69, lon: 97, nbnd: 2)
Coordinates:
  * member     (member) <U15 60B 'r10i1181p1f1'
  * lat        (lat) float32 276B 15.55 16.49 17.43 18.38 ... 77.75 78.69 79.63
    time_bnds  (time, nbnd) object 234kB dask.array<chunksize=(365, 2), meta=np.ndarray>
  * time       (time) object 117kB 2060-01-01 12:00:00 ... 2099-12-31 12:00:00
  * lon        (lon) float32 388B -80.0 -78.75 -77.5 -76.25 ... 37.5 38.75 40.0
Dimensions without coordinates: nbnd
Data variables:
    u          (member, time, lat, lon) float32 391MB dask.array<chunksize=(1, 365, 69, 97), meta=np.ndarray>
    lev        (member, time, lat, lon) float32 391MB dask.array<chunksize=(1, 365, 69, 97), meta=np.ndarray>
    v          (member, time, lat, lon) float32 391MB dask.array<chunksize=(1, 365, 69, 97), meta=np.ndarray>
    s          (member, time, lat, lon) float32 391MB dask.array<chunksize=(1, 365, 69, 97), meta=np.ndarray>
Attributes: (12/20)
    Conventions:                       CF-1.0
    host:                              mom1
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  day_1
    ...                                ...
    intake_esm_attrs:start_time:       2015-01-01 12:00:00
    intake_esm_attrs:end_time:         2100-12-31 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.ssp370.daily.cmip6
    orig_lev:                          [197.90809 232.82862 273.91082 322.2419 ]
    flattened:                         1